In [24]:
!pip install openai

In [53]:
def read_secrets(config_file):
    with open(config_file, "r") as file:
        data = file.read()
        data = json.loads(data)
        file.close()
        for item in data:
            os.environ[item] = data[item]

read_secrets("secrets.json")

In [59]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent
#from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI
import requests
import json
import os

#### Nutritionix Request Test

In [60]:
url = 'https://trackapi.nutritionix.com/v2/natural/nutrients'
headers = {
    'Content-Type': 'application/json',
    'x-app-id': os.environ['APPLICATION_ID'],
    'x-app-key':os.environ['APPLICATION_KEY'],
  #  'x-remote-user-id': 0
  }
body = {
    "query": "egg omelette with three eggs"
  }
response = requests.post(url=url, headers=headers, json=body)


In [61]:
response.json()

{'foods': [{'food_name': 'egg omelette',
   'brand_name': None,
   'serving_qty': 1,
   'serving_unit': '3-egg omelet',
   'serving_weight_grams': 178.38,
   'nf_calories': 323.47,
   'nf_total_fat': 25.32,
   'nf_saturated_fat': 5.86,
   'nf_cholesterol': 624.96,
   'nf_sodium': 533.35,
   'nf_total_carbohydrate': 1.39,
   'nf_dietary_fiber': 0.07,
   'nf_sugars': 0.62,
   'nf_protein': 21.13,
   'nf_potassium': 235.72,
   'nf_p': 333.09,
   'full_nutrients': [{'attr_id': 203, 'value': 21.1307},
    {'attr_id': 204, 'value': 25.3197},
    {'attr_id': 205, 'value': 1.3935},
    {'attr_id': 207, 'value': 2.5526},
    {'attr_id': 208, 'value': 323.47},
    {'attr_id': 210, 'value': 0.0001},
    {'attr_id': 211, 'value': 0.6223},
    {'attr_id': 212, 'value': 0.0007},
    {'attr_id': 213, 'value': 0},
    {'attr_id': 214, 'value': 0},
    {'attr_id': 221, 'value': 0},
    {'attr_id': 255, 'value': 127.9693},
    {'attr_id': 262, 'value': 0},
    {'attr_id': 263, 'value': 0},
    {'attr_id

#### LLM Open AI

In [81]:
from getpass import getpass

OPENAI_API_KEY = getpass()

In [82]:
import os

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [54]:
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_openai_tools_agent

llm = ChatOpenAI()

### Agent Building

In [129]:
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool

# class FoodSearchInput(BaseModel):
#     query: str = Field(description="description of a single food")

@tool
def FoodLookup(query: str) -> dict:
    "Function to lookup food nutrion facts"
    url = 'https://trackapi.nutritionix.com/v2/natural/nutrients'
    headers = {
        'Content-Type': 'application/json',
        'x-app-id': os.environ['APPLICATION_ID'],
        'x-app-key':os.environ['APPLICATION_KEY'],
    #  'x-remote-user-id': 0
    }
    body = {
        "query": query
    }
    response = requests.post(url=url, headers=headers, json=body)
    return response.json()

# foodLookupTool = StructuredTool.from_function(
#     func=FoodLookup,
#     name="Food Lookup Tool",
#     description="lookup the nutrition information of a specfic food",
#     args_schema=FoodSearchInput,
#     # coroutine= ... <- you can specify an async method if desired as well
# )

In [130]:
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.prompts import MessagesPlaceholder

system_prompt = (
    "You are a helpful AI bot."
    "That is skilled at calculating calories and nutrion facts based on text input."
    "You have a tool that is able to find caloric information based on a string query."
    "Your job is to return the total calories and grams of protein consumed"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "Today I ate {food_query}. What is the total calories and protein in it?"),
    MessagesPlaceholder("agent_scratchpad")
    
])

tools = [FoodLookup]

agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


In [131]:
agent_executor.invoke(input={'food_query': "One cup, Chobani Greek Yogurt, 1/2 cup Granola"})



> Entering new AgentExecutor chain...

Invoking: `FoodLookup` with `{'query': 'One cup Chobani Greek Yogurt'}`


{'foods': [{'food_name': 'greek yogurt', 'brand_name': None, 'serving_qty': 1, 'serving_unit': 'cup', 'serving_weight_grams': 226.67, 'nf_calories': 133.74, 'nf_total_fat': 0.88, 'nf_saturated_fat': 0.27, 'nf_cholesterol': 11.33, 'nf_sodium': 81.6, 'nf_total_carbohydrate': 8.16, 'nf_dietary_fiber': 0, 'nf_sugars': 7.34, 'nf_protein': 23.1, 'nf_potassium': 319.6, 'nf_p': 306, 'full_nutrients': [{'attr_id': 203, 'value': 23.0977}, {'attr_id': 204, 'value': 0.884}, {'attr_id': 205, 'value': 8.1601}, {'attr_id': 207, 'value': 1.632}, {'attr_id': 208, 'value': 133.7353}, {'attr_id': 210, 'value': 0}, {'attr_id': 211, 'value': 0}, {'attr_id': 212, 'value': 0}, {'attr_id': 213, 'value': 5.7574}, {'attr_id': 214, 'value': 0}, {'attr_id': 221, 'value': 0}, {'attr_id': 255, 'value': 192.8962}, {'attr_id': 262, 'value': 0}, {'attr_id': 263, 'value': 0}, {'attr_id': 268, 'value': 557.

{'food_query': 'One cup, Chobani Greek Yogurt, 1/2 cup Granola',
 'output': 'The total calories and protein in the One cup of Chobani Greek Yogurt and 1/2 cup of Granola are as follows:\n\n- Chobani Greek Yogurt (One cup):\n  - Calories: 133.74\n  - Protein: 23.1 grams\n\n- Granola (1/2 cup):\n  - Calories: 298.29\n  - Protein: 8.34 grams\n\nTotal Calories: 133.74 + 298.29 = 432.03 calories\nTotal Protein: 23.1 + 8.34 = 31.44 grams\n\nTherefore, the total calories consumed are 432.03 calories and the total protein consumed is 31.44 grams.'}

In [31]:
import requests
import json

url = "http://127.0.0.1:8001/aifitnessapp/query/invoke"
data = {
    "input": {
        "query": "Sourdough bread 2 slices plus 2 tablespoons of salted butter"
        }
}

response = requests.post(url=url, json=data)

In [35]:
!pip install fastapi

  Using cached fastapi-0.111.0-py3-none-any.whl.metadata (25 kB)
  Using cached starlette-0.37.2-py3-none-any.whl.metadata (5.9 kB)
  Using cached fastapi_cli-0.0.4-py3-none-any.whl.metadata (7.0 kB)
  Using cached jinja2-3.1.4-py3-none-any.whl.metadata (2.6 kB)
  Using cached python_multipart-0.0.9-py3-none-any.whl.metadata (2.5 kB)
  Using cached uvicorn-0.30.1-py3-none-any.whl.metadata (6.3 kB)
  Using cached dnspython-2.6.1-py3-none-any.whl.metadata (5.8 kB)
  Using cached typer-0.12.3-py3-none-any.whl.metadata (15 kB)
  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
  Using cached shellingham-1.5.4-py2.py3-none-any.whl.metadata (3.5 kB)
  Using cached rich-13.7.1-py3-none-any.whl.metadata (18 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
Using cached fastapi-0.111.0-py3-none-any.whl (91 kB)
Using cached fastapi

In [45]:
from langserve import RemoteRunnable
agent = RemoteRunnable("http://127.0.0.1:8001/aifitnessapp/query/")
response = agent.invoke({
        "query": "Sourdough bread 2 slices"
        })

HTTPStatusError: Server error '500 Internal Server Error' for url 'http://127.0.0.1:8001/aifitnessapp/query/invoke'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/500 for Internal Server Error

In [48]:
from typing import Sequence, List

from langchain_core.language_models import BaseLanguageModel
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_core.runnables import Runnable, RunnablePassthrough
from langchain_core.tools import BaseTool
from langchain_core.utils.function_calling import convert_to_openai_tool

from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from pydantic.v1 import BaseModel

class Response(BaseModel):
    """Final response to the question being asked"""

    answer: str = Field(description="The final answer to respond to the user")
    sources: List[int] = Field(
        description="List of page chunks that contain answer to the question. Only include a page chunk if it contains relevant information"
    )

def parse(output):
    # If no function was invoked, return to user
    if "function_call" not in output.additional_kwargs:
        return AgentFinish(return_values={"output": output.content}, log=output.content)

    # Parse out the function call
    function_call = output.additional_kwargs["function_call"]
    name = function_call["name"]
    inputs = json.loads(function_call["arguments"])

    # If the Response function was invoked, return to the user with the function inputs
    if name == "Response":
        return AgentFinish(return_values=inputs, log=str(function_call))
    # Otherwise, return an agent action
    else:
        return AgentActionMessageLog(
            tool=name, tool_input=inputs, log="", message_log=[output]
        )

def create_openai_tools_agent_custom_output(
    llm: BaseLanguageModel, tools: Sequence[BaseTool], prompt: ChatPromptTemplate, output_parser
) -> Runnable:
    llm_with_tools = llm.bind(tools=[convert_to_openai_tool(tool) for tool in tools])

    agent = (
        RunnablePassthrough.assign(
            agent_scratchpad=lambda x: format_to_openai_tool_messages(
                x["intermediate_steps"]
            )
        )
        | prompt
        | llm_with_tools
        | output_parser
    )
    return agent

In [49]:
from pydantic.v1 import BaseModel, Field
from langchain.tools import StructuredTool
import requests
import os

class FoodLookupToolInput(BaseModel):
    query: str = Field(description="The food you wish to look up")

def FoodLookup(query: str):
    "Function to lookup food nutrion facts"
    url = 'https://trackapi.nutritionix.com/v2/natural/nutrients'
    headers = {
        'Content-Type': 'application/json',
        'x-app-id': os.environ['APPLICATION_ID'],
        'x-app-key':os.environ['APPLICATION_KEY'],
    #  'x-remote-user-id': 0
    }
    body = {
        "query": query
    }
    response = requests.post(url=url, headers=headers, json=body)
    return response.json()

food_lookup_tool = StructuredTool.from_function(
        func=FoodLookup,
        name="food_lookup_tool",
        description="Food Lookup Tool",
        args_schema=FoodLookupToolInput,
        
    )

In [51]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

tools = [food_lookup_tool, Response]

In [ ]:
agent = create_openai_tools_agent_custom_output(llm, )